# 📝 3. N-gram 언어 모델로 문장 생성하기

지난 시간에 배웠던 **Bigram(2-gram)** 빈도수를 활용해서, 컴퓨터가 자동으로 문장을 만들게 해볼 거야.
원리는 아주 간단해! **"끝말잇기"**랑 비슷한데, 그냥 아무 말이나 잇는 게 아니라 **"통계적으로 가장 자연스러운 단어"**를 확률적으로 고르는 거지.

1. **현재 단어**를 본다.
2. 그 단어 뒤에 **자주 나왔던 단어들**을 후보로 둔다.
3. 빈도수가 높을수록 뽑힐 확률을 높게 설정해서 **다음 단어를 뽑는다.**
4. 이걸 반복한다!

In [ ]:
import nltk
from nltk.util import ngrams
from collections import Counter
import random

# 필요한 라이브러리들을 가져오는 단계야!

### 1단계: 데이터 준비 (Training Data)
먼저 모델에게 "한국어 문장 패턴"을 가르쳐줄 예시 문장들을 준비하자.

In [ ]:
train_text = "자연어 처리는 재미있다. 자연어 처리는 어렵지만 도전하고 싶다. 오늘은 날씨가 좋다."

# 1. 토큰화 (단어 단위로 자르기)
train_tokens = nltk.word_tokenize(train_text)

# 2. Bigram 만들기 (앞뒤 단어 짝짓기)
# 예: ('자연어', '처리는'), ('처리는', '재미있다') ...
bigram = list(ngrams(train_tokens, 2))

print("사용된 토큰들(Unigram):", train_tokens[:5], "...") # 5개만 출력
print("생성된 짝꿍들(Bigram):", bigram[:5], "...")

### 2단계: 빈도수 계산 (Counting)
어떤 단어 짝꿍이 얼마나 자주 나왔는지 숫자를 세어보자. 이 숫자가 곧 **확률**이 될 거야.

In [ ]:
# Unigram(단어) 빈도 확인
unigram_freq = Counter(train_tokens)

# Bigram(짝꿍) 빈도 확인
bigram_freq = Counter(bigram)

print("✅ '자연어'는 몇 번 나왔을까? ->", unigram_freq['자연어'])
print("✅ ('자연어', '처리는') 짝꿍은 몇 번 나왔을까? ->", bigram_freq[('자연어', '처리는')])

### 3단계: 문장 생성 함수 만들기 (핵심!)
이제 배운 통계 정보를 바탕으로 다음 단어를 예측해서 이어 붙이는 함수를 만들 거야.

In [ ]:
def generate_text_bigram(seed, unigram_freq, bigram_freq, max_len=10):
    """
    seed: 문장을 시작할 첫 단어
    unigram_freq: 단어 빈도수
    bigram_freq: 짝꿍 단어 빈도수
    max_len: 생성할 최대 문장 길이
    """
    current_word = seed
    generated = [current_word] # 생성된 단어들을 담을 리스트

    # 지정한 길이만큼 반복해서 다음 단어를 찾음
    for _ in range(max_len - 1):
        # 1. 2-gram 목록에서 '현재 단어(current_word)'로 시작하는 모든 짝꿍 찾기
        # 예: current_word가 '자연어'라면 -> ('자연어', '처리는') 등을 찾음
        # bigram_freq.items()는 ((단어1, 단어2), 빈도수) 형태를 가짐
        candidates = [(bigram, freq) for bigram, freq in bigram_freq.items() if bigram[0] == current_word]

        # 2. 만약 이어질 단어가 없다면? (학습한 적 없는 단어 등으로 끝남) -> 중단
        if not candidates:
            break

        # 3. 후보 단어들과 그 빈도수(가중치) 분리
        # candidates 예시: [ (('자연어', '처리는'), 2), ... ]
        # bigram[1]은 짝꿍 중 뒤에 오는 단어를 의미함
        words, freqs = zip(*[(bigram[1], freq) for bigram, freq in candidates])
        
        # 4. 확률 기반으로 다음 단어 랜덤 뽑기 (weights=freqs 이용)
        # 빈도수가 높을수록 뽑힐 확률이 높음!
        next_word = random.choices(words, weights=freqs)[0]
        
        # 5. 결과 리스트에 추가하고, 현재 단어를 업데이트
        generated.append(next_word)
        current_word = next_word
        
    return ' '.join(generated)

### 4단계: 실행해보기!
"자연어"라는 단어를 던져주면, 뒤에 어떤 말이 이어질까?

In [ ]:
# "자연어"로 시작하는 문장을 20단어 길이로 만들어줘
generated_sentence = generate_text_bigram("자연어", unigram_freq, bigram_freq, max_len=20)

print("🤖 생성된 문장:", generated_sentence)

### 🎓 퀴즈 (Quiz)

1. 우리가 만든 이 모델은 오직 **바로 직전의 (    )개의 단어**만 보고 다음 단어를 예측해. 빈칸에 들어갈 숫자는?
2. `random.choices`를 쓸 때 `weights`에 **빈도수**를 넣어주는 이유는, 많이 나온 단어를 더 ( **자주** / **적게** ) 뽑히게 하기 위해서다.